In [11]:
import numpy as np 
import pulp

def reusable_resource_allocation(initial_supply, supply, demand, adj_matrix,
                                  obj_dir='shortage', send_new_only=False,
                                  send_receive_switch_time=0, min_send_amt=0,
                                  smoothness_penalty=0, setup_cost=0,
                                  sent_penalty=0, verbose=False):
    import pulp
    import numpy as np

    N, T = supply.shape
    model = pulp.LpProblem("ResourceAllocation", pulp.LpMinimize)

    # Variables
    sent = [[[pulp.LpVariable(f"sent_{i}_{j}_{t}", lowBound=0)
              for t in range(T)] for j in range(N)] for i in range(N)]
    obj_dummy = [[pulp.LpVariable(f"obj_dummy_{i}_{t}", lowBound=0)
                  for t in range(T)] for i in range(N)]

    # Handle minimum send amount
    if min_send_amt > 0:
        for i in range(N):
            for j in range(N):
                for t in range(T):
                    model += sent[i][j][t] >= min_send_amt * (sent[i][j][t] > 0)

    # Constraints for resource transfer
    if send_new_only:
        for t in range(T):
            for i in range(N):
                model += pulp.lpSum(sent[i][j][t] for j in range(N)) <= max(0, supply[i, t])
    else:
        for t in range(T):
            for i in range(N):
                received = pulp.lpSum(sent[j][i][u] for j in range(N) for u in range(t+1))
                sent_out = pulp.lpSum(sent[i][j][u] for j in range(N) for u in range(t+1))
                model += received - sent_out <= initial_supply[i] + np.sum(supply[i, :t+1])

    # Adjacency matrix constraints
    for i in range(N):
        for j in range(N):
            if not adj_matrix[i, j]:
                model += pulp.lpSum(sent[i][j][t] for t in range(T)) == 0

    # Smoothness penalty handling
    if smoothness_penalty > 0:
        for i in range(N):
            for j in range(N):
                for t in range(1, T):
                    diff = pulp.LpVariable(f"diff_{i}_{j}_{t}", lowBound=0)
                    model += diff >= sent[i][j][t] - sent[i][j][t-1]
                    model += diff >= sent[i][j][t-1] - sent[i][j][t]
                    model += smoothness_penalty * diff

    # Objective Function
    model += pulp.lpSum([obj_dummy[i][t] for i in range(N) for t in range(T)]) \
             + sent_penalty * pulp.lpSum([sent[i][j][t] for i in range(N) for j in range(N) for t in range(T)])

    # Solve the model
    if verbose:
        model.solve()
    else:
        model.solve(pulp.PULP_CBC_CMD(msg=False))

    if model.status == pulp.LpStatusOptimal:
        print("Optimal solution found!")
        return { "status": "Optimal", "objective": pulp.value(model.objective) }
    else:
        print("Problem has no optimal solution.")
        return { "status": "Not solved", "objective": None }
    

# Example test case
initial_supply = np.array([10, 15, 20])
supply = np.array([[0, 0], [0, 0], [0, 0]])
demand = np.array([[8, 6], [25, 15], [10, 10]])
adj_matrix = np.array([[True, True, False], [True, True, True], [False, True, True]])

result = reusable_resource_allocation(initial_supply, supply, demand, adj_matrix, verbose=True)
print(result)


Optimal solution found!
{'status': 'Optimal', 'objective': 0.0}


In [13]:
initial_supply = np.array([10, 15, 20])
supply = np.array([[0, 0], [0, 0], [0, 0]])
demand = np.array([[8, 6], [25, 15], [10, 10]])
adj_matrix = np.array([[True, True, False], [True, True, True], [False, True, True]])


In [15]:
N, T = supply.shape
model = pulp.LpProblem("ResourceAllocation", pulp.LpMinimize)

# Variables
sent = [[[pulp.LpVariable(f"sent_{i}_{j}_{t}", lowBound=0)
            for t in range(T)] for j in range(N)] for i in range(N)]
obj_dummy = [[pulp.LpVariable(f"obj_dummy_{i}_{t}", lowBound=0)
                for t in range(T)] for i in range(N)]


In [74]:
import numpy as np 
import pulp

def reusable_resource_allocation(
        initial_supply,
        supply,
        demand,
        adj_matrix,
        obj_dir='shortage',
        send_new_only=False,
        send_receive_switch_time=1,
        min_send_amt=1,
        smoothness_penalty=1,
        setup_cost=1,
        sent_penalty=1,
        verbose=False):
    '''
        description: models a linear optimization problem to manage the allocation of reusable resources across different locations over time
    
        initial_supply: An array representing the initial quantity of resources available at each location at the beginning of the planning horizon.

        supply: A 2D array (matrix) where each element [i][t] represents the additional supply of resources that becomes available at location i at time period t.

        demand: A 2D array (matrix) where each element [i][t] indicates the demand for resources at location i at time period t.

        adj_matrix: A 2D Boolean array (matrix) where each element [i][j] is True if resources can be sent directly from location i to location j.
                    If the value is False, no resources can be transferred between these two locations.

        obj_dir (Objective Direction, default 'shortage'): A string that can either be 'shortage' or 'overflow'. If set to 'shortage', the objective is to 
                                                           minimize the unmet demand (shortage) at each location and time period. If set to 'overflow', the
                                                           objective could be adjusted to minimize any excess of resources (though, in the current setup, 
                                                           we focus only on shortage).

        send_new_only (default False): A Boolean flag indicating whether resources can only be sent from the current period’s supply. 
                                       If True, resources from initial or cumulative supplies cannot be used, restricting the transfers to what is 
                                       freshly supplied in each period.

        send_receive_switch_time (default 0): An integer indicating a delay or switch time between sending and receiving resources. 
                                              This would impose additional constraints to manage the timing of transfers, but it's not 
                                            fully implemented in the provided script.

        min_send_amt (default 0): A non-negative value that sets a minimum threshold for the amount of resources sent in any transfer. 
                                  If set to a value greater than zero, transfers below this amount are not allowed.

        smoothness_penalty (default 0): A non-negative value used as a multiplier to penalize changes in the amount of resources sent between 
                                        consecutive periods. This can be used to enforce a more stable or smooth allocation over time.

        setup_cost (default 0): A non-negative value representing the cost to set up a transfer of resources between locations. 
                                If positive, this adds a fixed cost for each pair of locations between which resources are transferred, 
                                encouraging fewer unique transfer routes.

        
        sent_penalty (default 0): A non-negative value that penalizes the total amount of resources sent. 
                                  This can be used to minimize the total volume of transfers across the network.

        verbose (default False): A Boolean flag that, when set to True, allows the solver's output to be printed to the console, 
                                 providing details on the optimization process.
    '''
    
    import pulp
    import numpy as np

    # get the number of nodes (healthcare centers) in the network: N 
    # get the number of time periods: T
    N, T = supply.shape
    
    # Model: indicates that the objective of the linear programming problem is to minimize the value of the objective function
    model = pulp.LpProblem("ResourceAllocation", pulp.LpMinimize)

    # Variables: define the decision variables for your model 
    
    # edges in the network define that hospital i can send resources to hopital j at time t; can be no less than 0 (non-negativity constraint)
    # constructs a matrix of variables 
    sent = [[[pulp.LpVariable(f"sent_{i}_{j}_{t}", lowBound=0) for t in range(T)]for j in range(N)] for i in range(N)]
    switch = [[[pulp.LpVariable(f"switch_{i}_{j}_{t}", cat=pulp.LpBinary) for t in range(T)] for j in range(N)] for i in range(N)]

    
    # placeholder dummy variables that can be used to add addtional constraints to our model
    obj_dummy = [[pulp.LpVariable(f"obj_dummy_{i}_{t}", lowBound=0)
                  for t in range(T)] for i in range(N)]
    
    ####################################
    # ADD THE CONSTRAINTS TO THE MODEL #
    ####################################
    # Handle minimum send amount
    # Decision variables

    if min_send_amt > 0:
        for i in range(N):
            for j in range(N):
                for t in range(T):
                    model += sent[i][j][t] >= min_send_amt * switch[i][j][t]
                    model += sent[i][j][t] <= 10000 * switch[i][j][t]

    # Constraints for resource transfer
    if send_new_only:
        for t in range(T):
            for i in range(N):
                # for a given time period t you can only send as much resources from node i 
                # as you just recived in time period t 
                model += pulp.lpSum(sent[i][j][t] for j in range(N)) <= max(0, supply[i, t])
    else:
        for t in range(T):
            for i in range(N):
                # resources received from hospitals i 
                received = pulp.lpSum(sent[j][i][u] for j in range(N) for u in range(t+1))
                # resources sent out from hospital i 
                sent_out = pulp.lpSum(sent[i][j][u] for j in range(N) for u in range(t+1))
                model += received - sent_out <= initial_supply[i] + np.sum(supply[i, :t+1])

    # Adjacency matrix constraints
    for i in range(N):
        for j in range(N):
            # if 0; i.e we do not have an edge between i and j 
            if not adj_matrix[i, j]:
                # make sure that the amount sent form i to j for t in range T is 0 
                model += pulp.lpSum(sent[i][j][t] for t in range(T)) == 0

    # Smoothness penalty handling
    # if smoothness_penalty > 0:
    #     for i in range(N):
    #         for j in range(N):
    #             for t in range(1, T):
    #                 diff = pulp.LpVariable(f"diff_{i}_{j}_{t}", lowBound=0)
    #                 model += diff >= sent[i][j][t] - sent[i][j][t-1]
    #                 model += diff >= sent[i][j][t-1] - sent[i][j][t]
    #                 model += smoothness_penalty * diff

    # Objective Function
    # modify obj_dummy if we want to add additional constraints
    #model += pulp.lpSum([obj_dummy[i][t] for i in range(N) for t in range(T)]) \
    #         + sent_penalty * pulp.lpSum([sent[i][j][t] for i in range(N) for j in range(N) for t in range(T)])
    
    model += pulp.lpSum([obj_dummy[i][t] for i in range(N) for t in range(T)]) \
             + sent_penalty * pulp.lpSum([sent[i][j][t] for i in range(N) for j in range(N) for t in range(T)])

    # Solve the model
    if verbose:
        model.solve()
    else:
        model.solve(pulp.PULP_CBC_CMD(msg=False))

    if model.status == pulp.LpStatusOptimal:
        print("Optimal solution found!")
        return { "status": "Optimal", "objective": pulp.value(model.objective) },model
    else:
        print("Problem has no optimal solution.")
        return { "status": "Not solved", "objective": None }

# # Example test case
initial_supply = np.array([5, 5, 5])
supply = np.array([[0, 0], [0, 0], [0, 0]])
demand = np.array([[8, 6], [25, 15], [10, 10]])
adj_matrix = np.array([[True, True, True], [True, True, True], [True, True, True]])

result,model = reusable_resource_allocation(initial_supply, supply, demand, adj_matrix, verbose=True)
print(result)

Optimal solution found!
{'status': 'Optimal', 'objective': 0.0}


In [75]:
# Access and print the values of decision variables
for i in range(N):
    for j in range(N):
        for t in range(T):
            print(f"sent[{i}][{j}][{t}] = {sent[i][j][t].varValue}")


sent[0][0][0] = None
sent[0][0][1] = None
sent[0][1][0] = None
sent[0][1][1] = None
sent[0][2][0] = None
sent[0][2][1] = None
sent[1][0][0] = None
sent[1][0][1] = None
sent[1][1][0] = None
sent[1][1][1] = None
sent[1][2][0] = None
sent[1][2][1] = None
sent[2][0][0] = None
sent[2][0][1] = None
sent[2][1][0] = None
sent[2][1][1] = None
sent[2][2][0] = None
sent[2][2][1] = None


In [76]:
model

ResourceAllocation:
MINIMIZE
1*obj_dummy_0_0 + 1*obj_dummy_0_1 + 1*obj_dummy_1_0 + 1*obj_dummy_1_1 + 1*obj_dummy_2_0 + 1*obj_dummy_2_1 + 1*sent_0_0_0 + 1*sent_0_0_1 + 1*sent_0_1_0 + 1*sent_0_1_1 + 1*sent_0_2_0 + 1*sent_0_2_1 + 1*sent_1_0_0 + 1*sent_1_0_1 + 1*sent_1_1_0 + 1*sent_1_1_1 + 1*sent_1_2_0 + 1*sent_1_2_1 + 1*sent_2_0_0 + 1*sent_2_0_1 + 1*sent_2_1_0 + 1*sent_2_1_1 + 1*sent_2_2_0 + 1*sent_2_2_1 + 0
SUBJECT TO
_C1: sent_0_0_0 - switch_0_0_0 >= 0

_C2: sent_0_0_0 - 10000 switch_0_0_0 <= 0

_C3: sent_0_0_1 - switch_0_0_1 >= 0

_C4: sent_0_0_1 - 10000 switch_0_0_1 <= 0

_C5: sent_0_1_0 - switch_0_1_0 >= 0

_C6: sent_0_1_0 - 10000 switch_0_1_0 <= 0

_C7: sent_0_1_1 - switch_0_1_1 >= 0

_C8: sent_0_1_1 - 10000 switch_0_1_1 <= 0

_C9: sent_0_2_0 - switch_0_2_0 >= 0

_C10: sent_0_2_0 - 10000 switch_0_2_0 <= 0

_C11: sent_0_2_1 - switch_0_2_1 >= 0

_C12: sent_0_2_1 - 10000 switch_0_2_1 <= 0

_C13: sent_1_0_0 - switch_1_0_0 >= 0

_C14: sent_1_0_0 - 10000 switch_1_0_0 <= 0

_C15: sent_1_0_

In [77]:
module ReusableResourceAllocation

using JuMP
using Gurobi

using LinearAlgebra
using MathOptInterface

export reusable_resource_allocation


function reusable_resource_allocation(
		initial_supply::Array{<:Real,1},
		supply::Array{<:Real,2},
		demand::Array{<:Real,2},
		adj_matrix::BitArray{2};
		obj_dir::Symbol=:shortage,
		send_new_only::Bool=false,
		sendrecieve_switch_time::Int=0,
		min_send_amt::Real=0,
		smoothness_penalty::Real=0,
		setup_cost::Real=0,
		sent_penalty::Real=0,
		verbose::Bool=false,
)
	N, T = size(supply)
	@assert(size(initial_supply, 1) == N)
	@assert(size(demand, 1) == N)
	@assert(size(demand, 2) == T)
	@assert(size(adj_matrix, 1) == N)
	@assert(size(adj_matrix, 2) == N)
	@assert(obj_dir in [:shortage, :overflow])

	model = Model(Gurobi.Optimizer)
	if !verbose set_silent(model) end

	@variable(model, sent[1:N,1:N,1:T])
	@variable(model, obj_dummy[1:N,1:T] >= 0)

	if min_send_amt <= 0
		@constraint(model, sent .>= 0)
	else
		@constraint(model, [i=1:N,j=1:N,t=1:T], sent[i,j,t] in MOI.Semicontinuous(Float64(min_send_amt), Inf))
	end

	objective = @expression(model, sum(obj_dummy))
	if sent_penalty > 0
		add_to_expression!(objective, sent_penalty*sum(sent))
	end
	if smoothness_penalty > 0
		@variable(model, smoothness_dummy[i=1:N,j=1:N,t=1:T-1] >= 0)
		@constraint(model, [t=1:T-1],  (sent[:,:,t] - sent[:,:,t+1]) .<= smoothness_dummy[:,:,t])
		@constraint(model, [t=1:T-1], -(sent[:,:,t] - sent[:,:,t+1]) .<= smoothness_dummy[:,:,t])

		add_to_expression!(objective, smoothness_penalty * sum(smoothness_dummy))
		add_to_expression!(objective, smoothness_penalty * sum(sent[:,:,1]))
	end
	if setup_cost > 0
		@variable(model, setup_dummy[i=1:N,j=i+1:N], Bin)
		@constraint(model, [i=1:N,j=i+1:N], [1-setup_dummy[i,j], sum(sent[i,j,:])+sum(sent[j,i,:])] in MOI.SOS1([1.0, 1.0]))
		add_to_expression!(objective, setup_cost*sum(setup_dummy))
	end
	@objective(model, Min, objective)

	if send_new_only
		@constraint(model, [t=1:T],
			sum(sent[:,:,t], dims=2) .<= max.(0, supply[:,t])
		)
	else
		@constraint(model, [i=1:N,t=1:T],
			sum(sent[i,:,t]) <=
				initial_supply[i]
				+ sum(supply[i,1:t])
				- sum(sent[i,:,1:t-1])
				+ sum(sent[:,i,1:t-1])
		)
	end

	for i = 1:N
		for j = 1:N
			if ~adj_matrix[i,j]
				@constraint(model, sum(sent[i,j,:]) .== 0)
			end
		end
	end

	if sendrecieve_switch_time > 0
		@constraint(model, [i=1:N,t=1:T-1],
			[sum(sent[:,i,t]), sum(sent[i,:,t:min(t+sendrecieve_switch_time,T)])] in MOI.SOS1([1.0, 1.0])
		)
		@constraint(model, [i=1:N,t=1:T-1],
			[sum(sent[:,i,t:min(t+sendrecieve_switch_time,T)]), sum(sent[i,:,t])] in MOI.SOS1([1.0, 1.0])
		)
	end

	flip_sign = (obj_dir == :shortage) ? 1 : -1
	z1, z2 = (obj_dir == :shortage) ? (0, -1) : (-1, 0)
	@constraint(model, [i=1:N,t=1:T],
		obj_dummy[i,t] >= flip_sign * (
			demand[i,t] - (
				initial_supply[i]
				+ sum(supply[i,1:t])
				- sum(sent[i,:,1:t+z1])
				+ sum(sent[:,i,1:t+z2])
			)
		)
	)

	optimize!(model)
	return model
end

end;

SyntaxError: invalid syntax (1270149916.py, line 1)